In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import statsmodels.api as sm
import warnings

from datetime import datetime
from ta.trend import ema_indicator, stc, macd, trix, kst
from ta.volatility import bollinger_hband, bollinger_lband, ulcer_index
from ta.momentum import kama, ppo, roc, rsi, stochrsi, tsi


# Suppress warnings to keep the output clean
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
BTC = "BTC-USD"
ETH = "ETH-USD"
SPDR = "SPY"
END_DATE = datetime.today().strftime("%Y-%m-%d")

In [5]:
stocks = yf.Ticker(
    BTC
)

with warnings.catch_warnings():
    warnings.filterwarnings(action='ignore')
    btc = stocks.history(start='2022-09-12', end=END_DATE, interval='1h').drop(columns=['Dividends', 'Stock Splits']).reset_index()
    btc = btc.rename(columns={'Datetime': 'Date'})
    btc_old = pd.read_parquet("../data/btc-2015-2023.pq").drop(columns=['Volume USD'])
    btc_old['Date'] = pd.to_datetime(btc_old['Date']).dt.tz_localize('UTC')
    btc_old = btc_old.loc[btc_old['Date'] < pd.to_datetime("2022-09-12 00:00:00+00:00")]

    btc_2019 = pd.read_parquet("../data/btc-2019.pq")
    btc_2019 = btc_2019.rename(columns={
        col: col.capitalize() for col in btc_2019.columns
    })
    btc_2019['Date'] = pd.to_datetime(btc_2019['Date']).dt.tz_localize('UTC')

    btc_2018 = pd.read_parquet("../data/btc-2018.pq")
    btc_2018 = btc_2018.rename(columns={
        col: col.capitalize() for col in btc_2018.columns
    })
    btc_2018['Date'] = pd.to_datetime(btc_2018['Date']).dt.tz_localize('UTC')


    btc_old = pd.concat([btc_old, btc_2018, btc_2019])

    def get_open(x: pd.Series):
        return x[x.index == np.min(x.index)]

    mapper = {
        'Open': get_open,
    }
    btc_old_processed = btc_old.sort_values(by='Date').reset_index(drop=True).groupby(pd.Grouper(key='Date', freq='H')).agg(mapper).reset_index()
    
    btc = pd.concat([btc, btc_old_processed], axis=0).sort_values(by='Date').reset_index(drop=True)[['Date', 'Open']]
    btc = btc.loc[(btc['Date'].dt.day_of_week == 0) & (btc['Date'].dt.hour == 20)]

# Trend indicators
btc['ema_12'] = ema_indicator(btc['Open'])
btc['ema_26'] = ema_indicator(btc['Open'], window=26)
btc['macd'] = macd(btc['Open'])
btc['kst'] = kst(btc['Open'])

# Volatility indicators
btc['bollinger_high'] = bollinger_hband(btc['Open'])
btc['bollinger_low'] = bollinger_lband(btc['Open'])
btc['ulcer_index'] = ulcer_index(btc['Open'])

# Momentum indicators
btc['kama'] = kama(btc['Open'])
btc['ppo'] = ppo(btc['Open'])
btc['roc'] = roc(btc['Open'])
btc['rsi'] = rsi(btc['Open'])
btc['stochrsi'] = stochrsi(btc['Open'])

btc['target'] = ((btc['Open'].shift(-1).fillna(np.nan).values - btc['Open'].values)/btc['Open'].values) * 100
btc['target_ema_2'] = ema_indicator(btc['target'], window=2)
btc['target_ema_4'] = ema_indicator(btc['target'], window=4)
btc['target_ma_2'] = btc['target'].shift(1).rolling(2).mean()
btc['target_ma_4'] = btc['target'].shift(1).rolling(4).mean()
# btc = btc.dropna()
btc

,Date,Open,ema_12,ema_26,macd,kst,bollinger_high,bollinger_low,ulcer_index,kama,ppo,roc,rsi,stochrsi,target,target_ema_2,target_ema_4,target_ma_2,target_ma_4
103,2015-10-12 20:00:00+00:00,247.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.056543,NaN,NaN,NaN,NaN
271,2015-10-19 20:00:00+00:00,272.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.954128,6.654933,NaN,NaN,NaN
439,2015-10-26 20:00:00+00:00,286.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.377622,17.136726,NaN,7.505336,NaN
607,2015-11-02 20:00:00+00:00,350.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.391429,11.973194,12.012809,13.665875,NaN
775,2015-11-09 20:00:00+00:00,382.87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-12.458537,-4.314626,2.224270,15.884525,11.694931
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75955,2024-06-10 20:00:00+00:00,69478.132812,65908.591840,58954.884042,6953.707798,840.015365,81177.653172,44598.044874,7.408726,63984.037777,11.794965,3.648718,65.798963,0.204373,-4.094589,-2.578201,-1.133269,0.200347,2.518919
76123,2024-06-17 20:00:00+00:00,66633.289062,66020.083720,59523.654785,6496.428936,794.096468,79964.049310,48174.914753,7.648734,64070.475618,10.914029,-6.141849,60.826736,0.001425,-10.623631,-7.941821,-4.929414,-1.833304,-1.226072
76291,2024-06-24 20:00:00+00:00,59554.414062,65025.365312,59525.933250,5499.432062,743.974072,77668.450521,52153.599869,8.700358,63961.611911,9.238716,-14.562561,50.583615,0.000000,6.139789,1.445919,-0.501733,-7.359110,-3.579382
76459,2024-07-01 20:00:00+00:00,63210.929688,64746.221369,59798.895949,4947.325421,692.853661,76283.343179,54905.006040,9.254372,63945.870454,8.273272,-11.883777,54.816224,0.161689,-10.620166,-6.598137,-4.549106,-2.241921,-2.037612
